In [ ]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import re
from tqdm import tqdm_notebook

# Data Load

In [ ]:
def create_df():

  #연도별 데이터 경로
  HTS_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS'
  Player_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수'
  Hitter_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본'
  #단일 데이터 경로
  Team_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_팀.csv'
  Schedule_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_경기일정_2021.csv'
  val_dir = '/content/drive/MyDrive/빅콘테스트/02_평가데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_평가데이터.xlsx'
  Crawl_dir = '/content/drive/MyDrive/빅콘테스트/crawling_data.csv'

  year = [2018, 2019, 2020, 2021]

  for i in year:
    globals()['HTS_{}'.format(i)]     = pd.read_csv(HTS_dir + '_' + str(i) + '.csv', encoding='CP949')
    globals()['Player_{}'.format(i)]  = pd.read_csv(Player_dir + '_' + str(i) + '.csv', encoding='CP949')
    globals()['Hitter_{}'.format(i)]  = pd.read_csv(Hitter_dir + '_' + str(i) + '.csv', encoding='CP949')

  global Team_df, Schedule_df, val_df, Crawl_df
  Team_df = pd.read_csv(Team_dir, encoding='CP949')
  Schedule_df = pd.read_csv(Schedule_dir, encoding='CP949')
  val_df = pd.read_excel(val_dir)
  Crawl_df = pd.read_csv(Crawl_dir, index_col=0)

######################

#연도별 데이터 하나로 통합
def concat_df():
  df_list = ['HTS', 'Player', 'Hitter']

#각 칼럼이 모두 같은지 검사 후 통합
  for i in df_list:
    if ((globals()['{}_2018'.format(i)].columns == globals()['{}_2019'.format(i)].columns).all()) and ((globals()['{}_2019'.format(i)].columns == globals()['{}_2020'.format(i)].columns).all()) and \
    ((globals()['{}_2020'.format(i)].columns == globals()['{}_2021'.format(i)].columns).all()):
      globals()['{}_df'.format(i)] = pd.concat( [ globals()['{}_2018'.format(i)], globals()['{}_2019'.format(i)], globals()['{}_2020'.format(i)], globals()['{}_2021'.format(i) ] ] )

#연도별 길이의 합이 통합된 데이터의 길이와 같은지 검사
  for i in df_list:
    if len(globals()['{}_2018'.format(i)]) + len(globals()['{}_2019'.format(i)]) + len(globals()['{}_2020'.format(i)]) + len(globals()['{}_2021'.format(i)]) == len(globals()['{}_df'.format(i)]):
      print('{} is complete!'.format(i))

######################

create_df()
concat_df()

HTS is complete!
Player is complete!
Hitter is complete!


In [ ]:
HTS_df.head()

,GYEAR,G_ID,PIT_ID,PCODE,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,STADIUM
0,2018,20180324HHWO0,180324_140436,62797,HH,1,131.50,42.7,플라이,149.59,고척
1,2018,20180324HHWO0,180324_140514,76753,HH,1,135.18,9.9,1루타,148.78,고척
2,2018,20180324HHWO0,180324_140647,71752,HH,1,152.41,2.1,1루타,148.59,고척
3,2018,20180324HHWO0,180324_140911,62700,HH,1,113.72,13.5,1루타,139.13,고척
4,2018,20180324HHWO0,180324_142050,68730,HH,2,54.11,16.4,번트안타,134.07,고척


In [ ]:
Crawl_df.head()

,GYEAR,PCODE,NAME,T_ID,POSITION,day,vs,inn,pitcher,batter,P,result,before,after,LEV,REa,WPs,WPe,WPa
0,2018,60100,백창수,HH,외,2018-03-27,NC,8초,이민호,백창수,1-0,유격수 땅볼,무사 6:9,1사 6:9,0.88,-0.258,93.100,95.3,-0.022
1,2018,60100,백창수,HH,외,2018-03-28,NC,2초,구창모,백창수,2-3,볼넷,2사 2루 0:2,"2사 1,2루 0:2",1.10,0.122,71.900,70.7,0.012
2,2018,60100,백창수,HH,외,2018-03-28,NC,4초,구창모,백창수,0-0,3루수 땅볼,2사 1:2,이닝종료 1:2,0.51,-0.117,67.400,68.8,-0.013
3,2018,60100,백창수,HH,외,2018-03-28,NC,7초,유원상,백창수,2-3,유격수 땅볼 실책,[실책포함],무사 2:2,무사 1루 2:2,1.540,0.398,50.0,44.100
4,2018,60100,백창수,HH,외,2018-03-30,SK,2말,김태훈,백창수,0-1,백창수/10473 : 2루수 직선타 아웃,"1사 2,3루 1:1","2사 2,3루 1:1",1.57,-0.833,64.100,56.1,-0.080


# DATE, INN 칼럼 추가 및 칼럼 정리

In [ ]:
HTS_df['DATE'] = HTS_df['G_ID'].apply(lambda x: int(x[:8]))
Crawl_df['DATE'] = Crawl_df['day'].apply(lambda x: int(x.replace('-', '')))
Crawl_df['INN'] = Crawl_df['inn'].apply(lambda x: int(x[0]) if len(x) == 2 else int(x[:2]))

HTS_col = ['DATE', 'PCODE', 'INN', 'HIT_VEL', 'HIT_ANG_VER', 'HIT_RESULT', 'PIT_VEL', 'PIT_ID', 'G_ID']
Crawl_col = ['DATE', 'PCODE', 'NAME', 'POSITION', 'INN', 'result', 'before', 'after', 'LEV', 'REa', 'WPs', 'WPe', 'WPa', 'P', 'pitcher', 'batter', 'vs']

HTS_df = HTS_df[HTS_col]
Crawl_df = Crawl_df[Crawl_col]

In [ ]:
HTS_df.head()

,DATE,PCODE,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,PIT_ID,G_ID
0,20180324,62797,1,131.50,42.7,플라이,149.59,180324_140436,20180324HHWO0
1,20180324,76753,1,135.18,9.9,1루타,148.78,180324_140514,20180324HHWO0
2,20180324,71752,1,152.41,2.1,1루타,148.59,180324_140647,20180324HHWO0
3,20180324,62700,1,113.72,13.5,1루타,139.13,180324_140911,20180324HHWO0
4,20180324,68730,2,54.11,16.4,번트안타,134.07,180324_142050,20180324HHWO0


In [ ]:
Crawl_df.head()

,DATE,PCODE,NAME,POSITION,INN,result,before,after,LEV,REa,WPs,WPe,WPa,P,pitcher,batter,vs
0,20180327,60100,백창수,외,8,유격수 땅볼,무사 6:9,1사 6:9,0.88,-0.258,93.100,95.3,-0.022,1-0,이민호,백창수,NC
1,20180328,60100,백창수,외,2,볼넷,2사 2루 0:2,"2사 1,2루 0:2",1.10,0.122,71.900,70.7,0.012,2-3,구창모,백창수,NC
2,20180328,60100,백창수,외,4,3루수 땅볼,2사 1:2,이닝종료 1:2,0.51,-0.117,67.400,68.8,-0.013,0-0,구창모,백창수,NC
3,20180328,60100,백창수,외,7,유격수 땅볼 실책,[실책포함],무사 2:2,무사 1루 2:2,1.540,0.398,50.0,44.100,2-3,유원상,백창수,NC
4,20180330,60100,백창수,외,2,백창수/10473 : 2루수 직선타 아웃,"1사 2,3루 1:1","2사 2,3루 1:1",1.57,-0.833,64.100,56.1,-0.080,0-1,김태훈,백창수,SK


# RESULT 통일

In [ ]:
HTS_df['HIT_RESULT'].value_counts()

플라이          36540
1루타          29235
땅볼아웃         26507
2루타           8146
홈런            4732
파울플라이         4097
직선타           3602
병살타           3085
내야안타(1루타)     1810
희생플라이         1603
3루타            672
인필드플라이         331
희생번트           239
야수선택            52
번트안타            48
번트아웃            44
삼중살타             2
Name: HIT_RESULT, dtype: int64

In [ ]:
Crawl_df['result'].value_counts()

볼넷                        18086
중견수 뜬공                    11866
2루수 땅볼                    11617
유격수 땅볼                    11404
중견수 안타                    11058
                          ...  
이대호/10251 : 투수 직선타 아웃         1
박정권/10096 : 유격수 직선타 아웃        1
오윤석/11397 : 중견수 직선타 아웃        1
박기혁/10241 : 우익수 직선타 아웃        1
김재현/10111 : 쓰리번트 아웃           1
Name: result, Length: 1990, dtype: int64

In [ ]:
delete_list = ['볼넷', '삼진', '몸에 맞는 볼', '고의4구', '스트라이크낫아웃 출루', '타구맞음', '타격방해', '[실책포함]']

HTS_df_clean = HTS_df[HTS_df['HIT_RESULT'].str.contains('번트') == False]
HTS_df_clean = HTS_df_clean[HTS_df_clean['HIT_RESULT'].str.contains('야수선택') == False]

Crawl_df_clean = Crawl_df[Crawl_df['result'].str.contains('번트') == False]
Crawl_df_clean = Crawl_df_clean[Crawl_df_clean['result'].str.contains('야수선택') == False]
Crawl_df_clean = Crawl_df_clean[Crawl_df_clean['result'].isin(delete_list) == False]
Crawl_df_clean = Crawl_df_clean[Crawl_df_clean['result'].str.contains('삼진') == False]

cd_result = Crawl_df_clean['result'].apply(lambda x : x.split(':')[1].strip() if len(x.split(':'))==2 else x.strip())
cd_result = cd_result.apply(lambda x : re.sub('1루수|2루수|3루수|유격수|포수|우익수|중견수|좌익수|투수', "", x).strip())
Crawl_df_clean.loc[:,['result']] = cd_result

In [ ]:
ground_list = ['땅볼아웃', '병살타', '삼중살타']
fly_list = ['플라이', '파울플라이', '희생플라이', '인필드플라이']

HTS_df_clean['RESULT_MG'] = HTS_df_clean['HIT_RESULT'].apply(lambda x: '땅볼' if x in ground_list else x)
HTS_df_clean['RESULT_MG'] = HTS_df_clean['RESULT_MG'].apply(lambda x: '플라이' if x in fly_list else x)
HTS_df_clean['RESULT_MG'] = HTS_df_clean['RESULT_MG'].apply(lambda x: '내야안타' if x == '내야안타(1루타)' else x)

HTS_df_clean['RESULT_MG'].value_counts()

플라이     42571
땅볼      29594
1루타     29235
2루타      8146
홈런       4732
직선타      3602
내야안타     1810
3루타       672
Name: RESULT_MG, dtype: int64

In [ ]:
ground_list = ['땅볼', '병살타', '땅볼 실책', '병살타 실책', '삼중살']
fly_list = ['뜬공', '파울 뜬공', '희생플라이', '인필드 뜬공', '플라이 실책', '희생플라이 실책']

Crawl_df_clean['RESULT_MG'] = Crawl_df_clean['result'].apply(lambda x: '땅볼' if x in ground_list else x)
Crawl_df_clean['RESULT_MG'] = Crawl_df_clean['RESULT_MG'].apply(lambda x: '플라이' if x in fly_list else x)
Crawl_df_clean['RESULT_MG'] = Crawl_df_clean['RESULT_MG'].apply(lambda x: '1루타' if x == '안타' else x)
Crawl_df_clean['RESULT_MG'] = Crawl_df_clean['RESULT_MG'].apply(lambda x: '홈런' if x == '그라운드 홈런' else x)
Crawl_df_clean['RESULT_MG'] = Crawl_df_clean['RESULT_MG'].apply(lambda x: '직선타' if '직선타' in x else x)

Crawl_df_clean['RESULT_MG'].value_counts()

땅볼      46972
플라이     44918
1루타     31498
2루타      8683
홈런       4956
직선타      3865
내야안타     3159
3루타       701
Name: RESULT_MG, dtype: int64

# 에러 검사

In [ ]:
tmp = HTS_df.drop_duplicates(['PCODE'], keep='first')['PCODE'].reset_index(drop=True).to_list()
tmp2 = Crawl_df.drop_duplicates(['PCODE'], keep='first')['PCODE'].reset_index(drop=True).to_list()

tmp3 = []
for i in tmp2:
    if i not in tmp:
        tmp3.append(i)

pitcher_sel_idx = []
for i in range(len(Crawl_df)):
    if Crawl_df['batter'][i] != Crawl_df['NAME'][i]:
        pitcher_sel_idx.append(i)

tmp4 = []
for i in tmp:
    if i not in tmp2:
        tmp4.append(i)

print('크롤링데이터에는 있고, HTS데이터에는 없는 PCODE: {}개\n크롤링데이터에는 없고, HTS데이터에는 있는 PCODE: {}개\nbatter이름이 최종 선택되지 않은 row: {}개'.format(len(tmp3), len(tmp4), len(pitcher_sel_idx)))

tmp5 = Player_df[Player_df['PCODE'].isin(tmp4)].drop_duplicates(['PCODE'], keep='first').reset_index(drop=True)
except_pcode = list(tmp5.PCODE)

print('\n크롤링데이터에 없고, HTS데이터에만 있는 PCODE 리스트 : {}'.format(except_pcode))

del tmp, tmp2, tmp3, tmp4, tmp5

크롤링데이터에는 있고, HTS데이터에는 없는 PCODE: 13개
크롤링데이터에는 없고, HTS데이터에는 있는 PCODE: 4개
batter이름이 최종 선택되지 않은 row: 0개

크롤링데이터에 없고, HTS데이터에만 있는 PCODE 리스트 : [64944, 67539, 77462, 63636]


In [ ]:
Player_df[Player_df['PCODE'].isin(except_pcode)]

,GYEAR,PCODE,NAME,T_ID,POSITION,AGE_VA,MONEY
214,2018,64944,박광열,NC,포,22,3600만원
343,2018,67539,나종덕,LT,포,20,2800만원
571,2018,77462,김동욱,KT,내,30,4000만원
133,2019,63636,고장혁,HT,내,29,5000만원
329,2019,67539,나종덕,LT,포,21,4300만원
197,2020,63636,고장혁,HT,내,30,5000만원
416,2020,67539,나균안,LT,투,22,4300만원
437,2021,67539,나균안,LT,투,23,4300만원


In [ ]:
origin_len = len(HTS_df_clean)

HTS_df_clean = HTS_df_clean[HTS_df_clean['PCODE'].isin(except_pcode) == False].reset_index(drop=True)
print('삭제된 row 개수 : {}개'.format(origin_len - len(HTS_df_clean)))

삭제된 row 개수 : 206개


In [ ]:
len(HTS_df_clean), len(Crawl_df_clean)

(120156, 144752)

# 중복 데이터 순서칼럼 추가

In [ ]:
HTS_dup = HTS_df_clean[HTS_df_clean.duplicated(['DATE', 'INN', 'PCODE', 'RESULT_MG', 'G_ID'], keep=False)].sort_values(['DATE', 'PCODE', 'PIT_ID'])
HTS_dup_idx = HTS_dup.index

HTS_order_list = [1,2] * int(len(HTS_dup)/2)
HTS_dup['order'] = HTS_order_list

zero = np.zeros(len(HTS_df_clean))
HTS_df_clean['order'] = zero

for i in range(len(HTS_dup)):
    idx = HTS_dup_idx[i]
    HTS_df_clean.loc[idx, 'order'] = HTS_order_list[i]

In [ ]:
Crawl_dup = Crawl_df_clean[Crawl_df_clean.duplicated(['DATE', 'INN', 'PCODE', 'RESULT_MG'], keep=False)].sort_values(['DATE', 'PCODE'])
Crawl_dup_idx = Crawl_dup.index

Crawl_order_list = [1,2] * int(len(Crawl_dup)/2)
Crawl_dup['order'] = Crawl_order_list

zero = np.zeros(len(Crawl_df_clean))
Crawl_df_clean['order'] = zero

for i in range(len(Crawl_dup)):
    idx = Crawl_dup_idx[i]
    Crawl_df_clean.loc[idx, 'order'] = Crawl_order_list[i]

In [ ]:
test_df = pd.merge(HTS_df_clean, Crawl_df_clean, how='left', on=['DATE', 'INN', 'PCODE', 'RESULT_MG', 'order'])

In [ ]:
test_df.head()

,DATE,PCODE,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,PIT_ID,G_ID,RESULT_MG,order,NAME,POSITION,result,before,after,LEV,REa,WPs,WPe,WPa,P,pitcher,batter,vs
0,20180324,62797,1,131.50,42.7,플라이,149.59,180324_140436,20180324HHWO0,플라이,0.0,양성우,외,뜬공,1사 0:0,2사 0:0,0.62,-0.180,52.2,53.8,-0.016,1-2,로저스,양성우,넥센
1,20180324,76753,1,135.18,9.9,1루타,148.78,180324_140514,20180324HHWO0,1루타,0.0,송광민,내,안타,2사 0:0,2사 1루 0:0,0.40,0.134,53.8,52.6,0.012,0-0,로저스,송광민,넥센
2,20180324,71752,1,152.41,2.1,1루타,148.59,180324_140647,20180324HHWO0,1루타,0.0,김태균,내,안타,2사 1루 0:0,"2사 1,2루 0:0",0.79,0.215,52.6,50.6,0.020,0-2,로저스,김태균,넥센
3,20180324,62700,1,113.72,13.5,1루타,139.13,180324_140911,20180324HHWO0,1루타,0.0,하주석,내,안타,"2사 1,2루 0:0","2사 1,2루 1:0",1.62,1.000,50.6,40.7,0.099,2-1,로저스,하주석,넥센
4,20180324,78288,2,161.90,3.5,1루타,145.75,180324_142310,20180324HHWO0,1루타,0.0,최재훈,포,안타,무사 2루 1:0,"무사 1,3루 1:0",1.13,0.715,33.8,27.9,0.059,0-2,로저스,최재훈,넥센


# 결측값 찾기

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120156 entries, 0 to 120155
Data columns (total 25 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   DATE         120156 non-null  int64  
 1   PCODE        120156 non-null  int64  
 2   INN          120156 non-null  int64  
 3   HIT_VEL      120156 non-null  float64
 4   HIT_ANG_VER  120156 non-null  float64
 5   HIT_RESULT   120156 non-null  object 
 6   PIT_VEL      120156 non-null  float64
 7   PIT_ID       120156 non-null  object 
 8   G_ID         120156 non-null  object 
 9   RESULT_MG    120156 non-null  object 
 10  order        120156 non-null  float64
 11  NAME         119504 non-null  object 
 12  POSITION     119504 non-null  object 
 13  result       119504 non-null  object 
 14  before       119504 non-null  object 
 15  after        119504 non-null  object 
 16  LEV          119504 non-null  object 
 17  REa          119504 non-null  float64
 18  WPs          119504 non-

In [ ]:
notnull_df = test_df[test_df['P'].isnull() == False].reset_index(drop=True)
null_df = test_df[test_df['P'].isnull()].reset_index(drop=True)

In [ ]:
null_df.PCODE.value_counts()

# 스탯티즈 및 HTS 누락 데이터 -> 삭제

60558    220
62537     44
67304     10
62415     10
76267      9
        ... 
63088      1
64153      1
66203      1
66710      1
73213      1
Name: PCODE, Length: 151, dtype: int64

# 실책포함 row WPa 수정

In [ ]:
error_df = notnull_df[notnull_df['before'].str.contains('실책')]
error_df = error_df.drop('before', axis=1)
error_df.rename(columns={'after':'before', 'LEV':'after', 'REa':'LEV', 'WPs':'REa', 'WPe':'WPs', 'WPa':'WPe'}, inplace=True)

error_df['WPa'] = (error_df['WPs'] - error_df['WPe'])/100

In [ ]:
error_df = error_df[['DATE', 'PCODE', 'INN', 'HIT_VEL', 'HIT_ANG_VER', 'HIT_RESULT',
       'PIT_VEL', 'PIT_ID', 'G_ID', 'RESULT_MG', 'order', 'NAME', 'POSITION',
       'result', 'before', 'after', 'LEV', 'REa', 'WPs', 'WPe', 'WPa', 'P', 'pitcher',
       'batter', 'vs']]

In [ ]:
for i in tqdm_notebook(error_df.index):
    notnull_df.loc[i] = error_df.loc[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/2008 [00:00<?, ?it/s]

# 최종 저장

In [ ]:
notnull_df.to_csv('/content/drive/MyDrive/빅콘테스트/Bigcontest_df.csv')

---